In [2]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from pathlib import Path

In [25]:
def searchKeywords(driver, selector:str, keyword:str):
    search = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    search.click()
    search.send_keys(keyword)
    search.send_keys(Keys.RETURN)
    
def getTitle(source, selector_title):
    soup = BeautifulSoup(source, 'lxml')
    titles_kor = soup.select(selector_title)

    return titles_kor

def appendTitles(list:list, titles):
    for i in titles:
        list.append(i.text)

def closePopup(driver, selector_popup):
    popup = driver.find_elements(By.CSS_SELECTOR, selector_popup)
    for i in range(len(popup)):
        popup[i].click()
    print(f"{len(popup)} Popup closed.")

def checkBlank(list: list):
    return "" if len(list) ==0 else list.text

def post_processing_text(text):
    return text.replace('\n', '').replace('\t', '') if text is not None else text

In [51]:
def kiss_crawling(keyword):

    driver = wd.Chrome(service=Service(ChromeDriverManager().install()))

    crawling_result_list = []
    driver.get("https://kiss.kstudy.com")

    # popup
    selector_popup_close = "div.pop_main_wrap > div.foot > div > div:nth-child(3) > a"
    closePopup(driver, selector_popup_close)

    # Search with Keywords
    selector_search = "#query"
    searchKeywords(driver, selector_search, keyword)

    selector_page = "#pagination > li.page-item > a"
    next = 1
    # title_list = []
    urls_list = []

    # Pagination  & crawling urls
    while True:
        # pages = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector_page))) 
        time.sleep(2)
        pages = driver.find_elements(By.CSS_SELECTOR, selector_page)
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'lxml')
        selector_title = "div.title > a"
        urls = [url + "/DetailOa/Ar?key=" + a["href"][29:37] for a in soup.select(selector_title) ]
        urls_list.extend(urls)
        # titles = getTitle(html_source, selector_title)
        # appendTitles(title_list, titles)
        if pages[-1].get_attribute("id") != "next": # Last page
            pages[-1].click()
            # titles = getTitle(html_source, selector_title)
            # appendTitles(title_list, titles)
            selector_title = "div.title > a"
            urls = [url + "/DetailOa/Ar?key=" + a["href"][29:37] for a in soup.select(selector_title) ]
            urls_list.extend(urls)
            print("Crawling URLs Done!")
            driver.quit()
            break
        elif pages[next].get_attribute("id") == "next": # prev, next
            pages[next].click()
            next = 2
        else:
            pages[next].click()
            next += 1
            
    html_sources = []
    crawling_result_list = []
    for url in tqdm(urls_list, desc="Crawling HTML sources"):
        driver = wd.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(url)
        html_source = driver.page_source
        html_sources.append(html_source)
    print("Crawling source finishhed!")

    for source in tqdm(html_sources, desc="Crawling abstracts"):
        selector_title = "div.title"
        selector_author = "div.author.mb-1"
        selector_journal = "div.mb-1 > a > span.value"
        selector_year = "div.mb-1 > span.value"
        selector_doi = "div.col-12 > div.mb-1 > span.value.color_font_gray"
        selector_abstract = "div#collapse-part1 > div.wrap_part_body"
        soup = BeautifulSoup(source, 'lxml')

        title_list = soup.select(selector_title)
        author_list = soup.select(selector_author)
        year_list = soup.select(selector_year)
        journal_list = soup.select(selector_journal)
        doi_list = soup.select(selector_doi)
        abstract_list = soup.select(selector_abstract)

        for title, author, journal, year, doi, abstract in zip(title_list, author_list, journal_list, year_list, doi_list, abstract_list):
            title_edit = post_processing_text(title.text)
            author_edit = post_processing_text(author.text)
            journal_edit = post_processing_text(journal.text)
            year_edit = post_processing_text(year.text)
            doi_edit = post_processing_text(doi.text)
            abstract_dict = {"title": title_edit, "author": author_edit, "journal": journal_edit,
                             "year": year_edit, "doi": doi_edit, "abstract": abstract.text}
            crawling_result_list.append(abstract_dict)  
    #  list to data.frame
    if ~Path("kiss/").exists():
        Path.mkdir("kiss/")
    df = pd.DataFrame(crawling_result_list)
    df.to_csv(f"kiss/kiss_{keyword}.csv")

In [52]:
kiss_crawling("pharmacist")

2 Popup closed.
Crawling URLs Done!


Processing items: 100%|██████████| 5/5 [00:35<00:00,  7.08s/it]


Crawling source finishhed!


100%|██████████| 5/5 [00:00<00:00, 51.87it/s]
